In [1]:
#Mounting Google drive.
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [12]:
#installing all the required packages.

!pip install --upgrade keras tensorflow
!pip install transformers
!pip install -U PyPDF2
!pip install python-docx
!pip install python-magic
!pip install transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 38.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=fa7cdf435e309d97731dd12484626fdfe86e654b890c92d6229e470024aabfa5
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# Data loading and preprocesing(clearing noice and removeing unwanted data(like special symbol, extra spaces, numbers))
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx
import magic
import matplotlib.pyplot as plt


BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/data.txt"
OUTPUT_FILE = "/content/gdrive/MyDrive/Final/Immigration/data.txt"

class DocumentManager:

    def __init__(self, base_path):
        """Initialize the DocumentManager with a base path."""
        self.base_path = base_path

    def detect_file_type(self, file_path):
        """Detect file type based on its content using python-magic and file extension."""

        # First, check based on file extension
        file_extension = os.path.splitext(file_path)[1].lower()

        if file_extension == '.pdf':
            return "application/pdf"
        elif file_extension == '.docx':
            return "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        elif file_extension == '.txt':
            return "text/plain"

        # If file extension is not recognized or matches, check using python-magic
        return magic.from_file(file_path, mime=True)

    def read_pdf(self, file_path):
        """Read a PDF file."""
        try:
            with open(file_path, "rb") as file:
                pdf_reader = PdfReader(file)
                text = "".join(page.extract_text() for page in pdf_reader.pages)
            return text
        except Exception as e:
            print(f"Error reading PDF {file_path}: {e}")
            raise e

    def read_word(self, file_path):
        """Read a DOCX file."""
        try:
            doc = docx.Document(file_path)
            text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
            return text
        except Exception as e:
            print(f"Error reading DOCX {file_path}: {e}")
            raise e

    def read_txt(self, file_path):
        """Read a TXT file."""
        try:
            with open(file_path, "r") as file:
                return file.read()
        except Exception as e:
            print(f"Error reading TXT {file_path}: {e}")
            raise e

    def compile_documents(self):
        """Compile documents from the base path into a single text file."""
        # Get file type
        file_type = self.detect_file_type(self.base_path)

        # Based on file type, read the content
        if file_type == "application/pdf":
            content = self.read_pdf(self.base_path)
        elif file_type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            content = self.read_word(self.base_path)
        elif file_type == "text/plain":
            content = self.read_txt(self.base_path)
        else:
            raise ValueError(f"Unsupported file type: {file_type}")

        # Clean and format the content
        formatted_content = self.clean_and_format_text(content)

        return formatted_content

    def clean_and_format_text(self, text):
        # Split the text into lines
        lines = text.splitlines()

        # Count lines before cleaning
        self.before_count = len(lines)

        # Filter out the empty or whitespace-only lines
        cleaned_lines = [line for line in lines if line.strip() != ""]

        # Count lines after cleaning
        self.after_count = len(cleaned_lines)

        # Join the cleaned lines and ensure no consecutive newlines
        formatted_text = "\n".join(cleaned_lines)
        formatted_text = re.sub(r'\n+', '\n', formatted_text).strip()

        return formatted_text

    def save_to_file(self, output_path):
        """Save the compiled documents to a specific file."""
        text_data = self.compile_documents()
        try:
            with open(output_path, "w") as f:
                f.write(text_data)
        except Exception as e:
            print(f"Error saving to file {output_path}: {e}")
            raise e

    def plot_results(self):
        plt.bar(['Before Cleaning', 'After Cleaning'], [self.before_count, self.after_count],color=['red', 'blue'])
        plt.title('Number of Records Before and After Cleaning')
        plt.ylabel('Number of Records')
        plt.show()

if __name__ == '__main__':
    doc_manager = DocumentManager(BASE_PATH)
    doc_manager.save_to_file(OUTPUT_FILE)
    print(f"Total Records Before Cleaning: {doc_manager.before_count}")
    print(f"Total Records After Cleaning: {doc_manager.after_count}")
    doc_manager.plot_results()


In [5]:
"""
Chatbot Dataset Cleaner
Semantic Filtering of User-Bot Conversations
"""
import json
import re
import numpy as np
import nltk
import torch
from sentence_transformers import SentenceTransformer, util

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/data.txt"
OUTPUT_FILE = "/content/gdrive/MyDrive/Final/Immigration/data.txt"

class DataCleaner:
    @staticmethod
    def is_valid_pattern(pattern):
        """Check if the pattern contains at least one word."""
        return len(pattern.strip()) > 0

    @staticmethod
    def is_valid_response(response):
        """Check if the response contains at least one word."""
        return len(response.strip()) > 0

    @staticmethod
    def batch_is_semantically_relevant(batch_patterns, batch_responses):
        """Determine if batches of patterns and responses are semantically relevant."""
        model = SentenceTransformer('bert-base-nli-mean-tokens')
        pattern_embeddings = model.encode(batch_patterns)
        response_embeddings = model.encode(batch_responses)
        similarity_scores = util.pytorch_cos_sim(pattern_embeddings, response_embeddings)
        similarity_threshold = 0.6
        return (similarity_scores >= similarity_threshold).any(dim=1).tolist()

    @classmethod
    def clean_data(cls, dataset_path):
        """Process and clean the dataset by filtering semantically irrelevant and duplicate pairs."""
        with open(dataset_path, 'r') as file:
            lines = file.readlines()
            patterns = [lines[i+1].strip() for i in range(0, len(lines), 2)]
            responses = [lines[i].strip() for i in range(0, len(lines), 2)]

        # Validate patterns and responses
        patterns = [pattern.split(': ')[1] for pattern in patterns if cls.is_valid_pattern(pattern)]
        responses = [response.split(': ')[1] for response in responses if cls.is_valid_response(response)]

        # Remove duplicates
        unique_pairs = list(set(zip(patterns, responses)))
        patterns = [item[0] for item in unique_pairs]
        responses = [item[1] for item in unique_pairs]

        filtered_patterns, filtered_responses = [], []
        for pattern, response in zip(patterns, responses):
            is_relevant = cls.batch_is_semantically_relevant([pattern], [response])
            if is_relevant[0]:
                filtered_patterns.append(pattern)
                filtered_responses.append(response)

        # Convert data to alternate user-bot format and save
        cleaned_data = []
        for p, r in zip(filtered_patterns, filtered_responses):
            cleaned_data.append(f"User: {p}\nBot: {r}\n")

        with open("/content/gdrive/MyDrive/Final/Immigration/data.txt", 'w') as outfile:
            outfile.writelines(cleaned_data)

        return 'train.txt'

    @staticmethod
    def compare_data(original_file, cleaned_file):
        """Compare the number of patterns and responses between original and cleaned data files."""
        with open(original_file, 'r') as orig_file:
            original_lines = orig_file.readlines()

        with open(cleaned_file, 'r') as cleaned_file:
            cleaned_lines = cleaned_file.readlines()

        print("Number of lines in original file:", len(original_lines))
        print("Number of lines in cleaned file:", len(cleaned_lines))


def main():
    dataset_path = "/content/gdrive/MyDrive/Project/Chatbot/traindata.txt"
    cleaned_file = DataCleaner.clean_data(dataset_path)
    DataCleaner.compare_data(dataset_path, cleaned_file)


if __name__ == "__main__":
    main()


In [6]:
# TEXT DATA PREPROCESING:
import re
import pickle
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import drive


# Check for GPU availability
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TensorFlow")

class ChatbotDataPreprocessor:
    def __init__(self, base_path, filepath, max_vocab=50000, max_seq_len=200):
        self.filepath = filepath
        self.max_vocab = max_vocab
        self.max_seq_len = max_seq_len
        self.base_path = base_path

        self.TOKENIZER_PATH = self.base_path + "tokenizer.pkl"
        self.SEQUENCES_PATH = self.base_path + "preprocessed_data.pkl"

        # Initialize tokenizer with special tokens and filters
        self.tokenizer = Tokenizer(num_words=self.max_vocab, oov_token='<OOV>', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
        self.questions = []
        self.answers = []

    # Load dialogue data from file
    def load_data(self):
        try:
            with open(self.filepath, 'r') as file:
                lines = file.readlines()
            if not lines:
                print("Warning: Loaded data is empty.")
                return []
            return lines
        except Exception as e:
            print(f"Error loading data from {self.filepath}: {e}")
            return []

    # Basic text cleaning
    def clean_text(self, text):
        text = text.lower().strip()
        text = re.sub(r"[^a-z0-9'\s]+", " ", text)
        return text

    # Extract question-answer pairs from loaded data
    def create_dialogue_pairs(self, lines):
        questions_temp = []
        answers_temp = []

        for index, line in enumerate(lines):
            if "User:" in line:
                questions_temp.append(self.clean_text(line.replace("User:", "")))
            elif "Bot:" in line:
                answers_temp.append(self.clean_text(line.replace("Bot:", "")))

        # Ensure that the number of questions and answers match
        min_len = min(len(questions_temp), len(answers_temp))
        self.questions = questions_temp[:min_len]
        self.answers = answers_temp[:min_len]

        if len(questions_temp) != len(answers_temp):
            print("Warning: Number of questions and answers don't match! Truncated to match lengths.")


    # Convert text data to padded numeric sequences
    def tokenize_and_pad(self, texts):
        sequences = self.tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(sequences, maxlen=self.max_seq_len, padding='post')
        return padded

    # Save tokenizer to file
    def save_tokenizer(self, filename):
        try:
            with open(filename, 'wb') as file:
                pickle.dump(self.tokenizer, file)
            print(f"Tokenizer saved successfully to {filename}.")
        except Exception as e:
            print(f"Error saving tokenizer to {filename}: {e}")

    # Save preprocessed data to file
    def save_sequences(self, filename, train_q, test_q, train_a, test_a):
        try:
            with open(filename, 'wb') as file:
                pickle.dump({
                    'train_questions': train_q,
                    'test_questions': test_q,
                    'train_answers': train_a,
                    'test_answers': test_a,
                    'tokenizer': self.tokenizer
                }, file)
            print(f"Sequences saved successfully to {filename}.")
        except Exception as e:
            print(f"Error saving sequences to {filename}: {e}")

    # Main preprocessing function
    def preprocess(self):
        lines = self.load_data()
        if not lines:
            return
        self.create_dialogue_pairs(lines)
        self.tokenizer.fit_on_texts(self.questions + self.answers)

        # Assign index 0 to the padding token after fitting the tokenizer on texts
        self.tokenizer.word_index['<pad>'] = 0

        train_q, test_q, train_a, test_a = train_test_split(self.questions, self.answers, test_size=0.2)

        train_q = self.tokenize_and_pad(train_q)
        test_q = self.tokenize_and_pad(test_q)
        train_a = self.tokenize_and_pad(train_a)
        test_a = self.tokenize_and_pad(test_a)

        self.save_tokenizer(self.TOKENIZER_PATH)
        self.save_sequences(self.SEQUENCES_PATH, train_q, test_q, train_a, test_a)



if __name__ == "__main__":
    BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/"
    FILE_PATH = BASE_PATH + "data.txt"

    preprocessor = ChatbotDataPreprocessor(BASE_PATH, filepath=FILE_PATH)
    preprocessor.preprocess()

In [7]:
#check the result from above preprocessing.

BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/"
with open(BASE_PATH + "preprocessed_data.pkl", "rb") as file:
    preprocessed_data = pickle.load(file)

print(preprocessed_data.keys())


In [8]:
#checking the fiels and basepath access
import os

BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/"

# Check if BASE_PATH exists
if not os.path.exists(BASE_PATH):
    raise ValueError(f"The directory {BASE_PATH} does not exist!")
else:
    print(f"{BASE_PATH} exists.")
    # List and print existing files inside BASE_PATH
    files_in_directory = os.listdir(BASE_PATH)
    if files_in_directory:
        print("Files inside the directory:")
        for file in files_in_directory:
            print(file)
    else:
        print("The directory is empty.")


/content/gdrive/MyDrive/Final/Immigration/ exists.
Files inside the directory:
data.txt
requirements.txt.gdoc
tokenizer.pkl
preprocessed_data.pkl
GPT
chatbot_model_best.pth
chatbot_model_final.pth
HotelAssist.ipynb


In [9]:
# !pip freeze > requirements.txt


In [10]:
# IMPORTS
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from keras.callbacks import EarlyStopping
from torch.utils.data import random_split
import numpy as np

# BASE CONFIGURATIONS
BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ENCODER
class Encoder(nn.Module):
    """Encoder module for the Seq2Seq model."""
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

# DECODER
class Decoder(nn.Module):
    """Decoder module for the Seq2Seq model."""
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim * 2, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

# SEQ2SEQ MODEL
class Seq2Seq(nn.Module):
    """Main Seq2Seq model integrating the Encoder and Decoder."""
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_hidden, encoder_cell = self.encoder(src)

        if self.encoder.rnn.bidirectional:
            hidden = torch.cat((encoder_hidden[0:encoder_hidden.size(0):2],
                                encoder_hidden[1:encoder_hidden.size(0):2]), dim=2)
            cell = torch.cat((encoder_cell[0:encoder_cell.size(0):2],
                              encoder_cell[1:encoder_cell.size(0):2]), dim=2)
        else:
            hidden, cell = encoder_hidden, encoder_cell

        input = trg[0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

# CHATBOT TRAINER
class ChatbotTrainer:
    def __init__(self, data_path,tokenizer):
        self.data_path = data_path
        with open(data_path, "rb") as file:
            preprocessed_data = pickle.load(file)
        # self.tokenizer = preprocessed_data['tokenizer']
        self.tokenizer = tokenizer
        self.questions = preprocessed_data['train_questions']
        self.answers = preprocessed_data['train_answers']
        self.input_dim = len(self.tokenizer.word_index) + 1
        self.output_dim = len(self.tokenizer.word_index) + 1
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    @staticmethod
    def _accuracy(preds, y):
        """Returns accuracy per batch."""
        _, predictions = torch.max(preds, 1)
        correct = (predictions == y).float()
        acc = correct.sum() / len(correct)
        return acc

    @staticmethod
    def _collate_fn(data):
        data.sort(key=lambda x: len(x[0]), reverse=True)
        src_seqs, trg_seqs = zip(*data)
        src_seqs = [torch.Tensor(s) for s in src_seqs]
        trg_seqs = [torch.Tensor(t) for t in trg_seqs]
        src_lengths = [len(s) for s in src_seqs]
        PAD_IDX = tokenizer.word_index['<pad>']  # Using <pad> token for padding
        # PAD_IDX = self.tokenizer.word_index['<pad>']
        src_padded = pad_sequence(src_seqs, padding_value=PAD_IDX, batch_first=True)
        trg_lengths = [len(t) for t in trg_seqs]
        trg_padded = pad_sequence(trg_seqs, padding_value=PAD_IDX, batch_first=True)
        return src_padded, src_lengths, trg_padded, trg_lengths

    def _train_epoch(self, model, dataloader, optimizer, criterion, clip, teacher_forcing_ratio):
        model.train()
        epoch_loss = 0
        epoch_acc = 0
        for i, (src, _, trg, _) in enumerate(dataloader):
            src, trg = src.long().to(DEVICE), trg.long().to(DEVICE)
            optimizer.zero_grad()
            output = model(src, trg, teacher_forcing_ratio)
            output_dim = output.shape[-1]
            output = output[1:].reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)
            loss = criterion(output, trg)
            acc = self._accuracy(output, trg)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

    def _evaluate(self, model, dataloader, criterion):
        model.eval()
        epoch_loss = 0
        epoch_acc = 0
        with torch.no_grad():
            for i, (src, _, trg, _) in enumerate(dataloader):
                src, trg = src.long().to(DEVICE), trg.long().to(DEVICE)
                output = model(src, trg, 0)
                output_dim = output.shape[-1]
                output = output[1:].reshape(-1, output_dim)
                trg = trg[1:].reshape(-1)
                loss = criterion(output, trg)
                acc = self._accuracy(output, trg)
                epoch_loss += loss.item()
                epoch_acc += acc.item()
        return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

    def train(self, model, train_data, valid_data, optimizer, criterion, n_epochs, clip, teacher_forcing_ratio, lr_scheduler):
        best_valid_loss = float('inf')
        epochs_without_improvement = 0  # Initialize the counter
        patience = 3

        for epoch in range(n_epochs):
            train_loss, train_acc = self._train_epoch(model, train_data, optimizer, criterion, clip, teacher_forcing_ratio)
            valid_loss, valid_acc = self._evaluate(model, valid_data, criterion)

            self.train_losses.append(train_loss)
            self.val_losses.append(valid_loss)
            self.train_accuracies.append(train_acc)
            self.val_accuracies.append(valid_acc)

            lr_scheduler.step(valid_loss)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                epochs_without_improvement = 0  # Reset the counter
                torch.save(model.state_dict(), BASE_PATH + 'chatbot_model_best.pth')
            else:
                epochs_without_improvement += 1  # Increment the counter

            print(f'Epoch: {epoch + 1:02}')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

            # Check if early stopping criterion is met
            if epochs_without_improvement >= patience:
                print("Early stopping triggered! No improvement for", epochs_without_improvement, "epochs.")
                break  # Stop training

        torch.save(model.state_dict(), BASE_PATH + 'chatbot_model_final.pth')


    def plot_results(self):
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(self.train_losses, label="Training Loss")
        plt.plot(self.val_losses, label="Validation Loss")
        plt.legend()
        plt.title("Loss vs. Epochs")

        plt.subplot(1, 2, 2)
        plt.plot(self.train_accuracies, label="Training Accuracy")
        plt.plot(self.val_accuracies, label="Validation Accuracy")
        plt.legend()
        plt.title("Accuracy vs. Epochs")
        plt.show()

import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pth', save_model=False):
        """
        Args:
            patience (int): Number of epochs to wait before stopping if no improvement is seen.
            verbose (bool): If True, prints a message for each validation loss improvement.
            delta (float): Minimum change in the validation loss to qualify as an improvement.
            path (str): Path for the checkpoint to be saved to.
            save_model (bool): If True, saves the entire model. Otherwise, only the state dict.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.save_model = save_model

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        if self.save_model:
            torch.save(model, self.path)
        else:
            torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


# --- SCRIPT/MAIN ----
if __name__ == "__main__":
    # Configuration
    BATCH_SIZE = 8
    EMB_DIM = 256
    HID_DIM = 1024
    N_LAYERS = 10
    DROPOUT = 0.2
    N_EPOCHS = 100
    CLIP = 1
    TEACHER_FORCING_RATIO = 0.5
    LEARNING_RATE = 0.0001
    train_ratio = 0.8

    with open(BASE_PATH + "preprocessed_data.pkl", "rb") as file:
        preprocessed_data = pickle.load(file)
    tokenizer = preprocessed_data['tokenizer']

    # Initialize the EarlyStopping class
    early_stopping = EarlyStopping(patience=3, verbose=True, path='chatbot_model_early_stopping_checkpoint.pth')

    chatbot_trainer = ChatbotTrainer(data_path=BASE_PATH + "preprocessed_data.pkl", tokenizer=tokenizer)

    # Splitting the dataset
    dataset = list(zip(chatbot_trainer.questions, chatbot_trainer.answers))
    train_size = int(train_ratio * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=chatbot_trainer._collate_fn)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=chatbot_trainer._collate_fn)

    # Model Initialization and Training
    enc = Encoder(chatbot_trainer.input_dim, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT).to(DEVICE)
    dec = Decoder(chatbot_trainer.output_dim, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT).to(DEVICE)
    model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    PAD_IDX = chatbot_trainer.tokenizer.word_index['<pad>']
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    # Train with early stopping
    # chatbot_trainer.train(model, train_dataloader, valid_dataloader, optimizer, criterion, N_EPOCHS, CLIP, TEACHER_FORCING_RATIO, lr_scheduler, early_stopping)
    # chatbot_trainer.train(model, train_dataloader, valid_dataloader, optimizer, criterion, N_EPOCHS, CLIP, TEACHER_FORCING_RATIO, lr_scheduler)
    # chatbot_trainer.plot_results()
    chatbot_trainer.train(model, train_dataloader, valid_dataloader, optimizer, criterion, N_EPOCHS, CLIP, TEACHER_FORCING_RATIO, lr_scheduler)
    chatbot_trainer.plot_results()

In [14]:
# Test the chatbot result
import torch
import pickle
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from keras.callbacks import EarlyStopping
from torch.utils.data import random_split
import numpy as np


# ENCODER
class Encoder(nn.Module):
    """Encoder module for the Seq2Seq model."""
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

# DECODER
class Decoder(nn.Module):
    """Decoder module for the Seq2Seq model."""
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim * 2, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

# SEQ2SEQ MODEL
class Seq2Seq(nn.Module):
    """Main Seq2Seq model integrating the Encoder and Decoder."""
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_hidden, encoder_cell = self.encoder(src)

        if self.encoder.rnn.bidirectional:
            hidden = torch.cat((encoder_hidden[0:encoder_hidden.size(0):2],
                                encoder_hidden[1:encoder_hidden.size(0):2]), dim=2)
            cell = torch.cat((encoder_cell[0:encoder_cell.size(0):2],
                              encoder_cell[1:encoder_cell.size(0):2]), dim=2)
        else:
            hidden, cell = encoder_hidden, encoder_cell

        input = trg[0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

class ChatbotResponder:
    def __init__(self, model_path, data_path, device):
        self.device = device
        with open(data_path, "rb") as file:
            preprocessed_data = pickle.load(file)
        self.tokenizer = preprocessed_data['tokenizer']
        self.input_dim = len(self.tokenizer.word_index) + 1
        self.output_dim = len(self.tokenizer.word_index) + 1
        self.PAD_IDX = self.tokenizer.word_index['<pad>']

        # Model Initialization
        EMB_DIM = 256
        HID_DIM = 512
        N_LAYERS = 2
        DROPOUT = 0.5
        self.enc = Encoder(self.input_dim, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT).to(self.device)
        self.dec = Decoder(self.output_dim, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT).to(self.device)
        self.model = Seq2Seq(self.enc, self.dec, self.device).to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()

    def _tokenize(self, sentence):
        tokens = self.tokenizer.texts_to_sequences([sentence])
        return torch.tensor(tokens).to(self.device)

    def _decode_tensor(self, tensor):
        decoded_words = []
        for tok_id in tensor:
            decoded_words.append(self.tokenizer.index_word[tok_id.item()])
            if tok_id.item() == self.tokenizer.word_index['<end>']:
                break
        return ' '.join(decoded_words[1:])

    def generate_response(self, user_input):
        src_tensor = self._tokenize(user_input)
        with torch.no_grad():
            hidden, cell = self.enc(src_tensor)
            trg_indexes = [self.tokenizer.word_index['<start>']]
            for i in range(100):
                trg_tensor = torch.tensor([trg_indexes[-1]]).to(self.device)
                output, hidden, cell = self.dec(trg_tensor, hidden, cell)
                pred_token = output.argmax(1).item()
                trg_indexes.append(pred_token)
                if pred_token == self.tokenizer.word_index['<end>']:
                    break
        trg_tokens = torch.tensor(trg_indexes, dtype=torch.long)
        response = self._decode_tensor(trg_tokens)
        return response

if __name__ == "__main__":
    model_path = "/content/gdrive/MyDrive/Final/Immigration/chatbot_model_final.pth"
    BASE_PATH = "/content/gdrive/MyDrive/Final/Immigration/"
    data_path = BASE_PATH + "preprocessed_data.pkl"
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    responder = ChatbotResponder(model_path, data_path, DEVICE)

    print("Chatbot is ready to chat! Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        response = responder.generate_response(user_input)
        print("Bot:", response)


AssertionError: ignored